Construct an image X with diagonal edges.

What happens if you apply the kernel K in this section to it?



In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from d2l import tensorflow as d2l


In [2]:
def corr2d(X, K):
    h, w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0]- h + 1 ,X.shape[1]- w + 1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j].assign(tf.reduce_sum(X[i : i+h, j : j+w] * K))
    return Y
            

In [3]:
X = tf.Variable(tf.ones((8, 6)))
X[2:6, :].assign(tf.zeros(X[ 2:6, :].shape))
X

<tf.Variable 'Variable:0' shape=(8, 6) dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [4]:
K = tf.constant([[1.0, -1.0]])

In [5]:
Y= corr2d(X, K)
Y

<tf.Variable 'Variable:0' shape=(8, 5) dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

What happens if you transpose X?



In [6]:
corr2d(tf.transpose(X), K)

<tf.Variable 'Variable:0' shape=(6, 7) dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]], dtype=float32)>

What happens if you transpose K?

In [7]:
corr2d(tf.transpose(X), tf.transpose(K))

<tf.Variable 'Variable:0' shape=(5, 8) dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

When you try to automatically find the gradient for the Conv2D class we created, what kind of error message do you see?

In [8]:
class Conv2d(keras.layers.Layer):
    def __init__(self, kernel_size):
        super().__init__()
        self.kernel_size = kernel_size
        initializer = tf.random_normal_initializer()
        self.weight = self.add_weight(name = 'w', shape = self.kernel_size, initializer = initializer)
        self.bias = self.add_weight(name = 'b', shape = (1,), initializer = initializer)
        
    def call(self , inputs):
        return corr2d(inputs, self.weight) + self.bias

conv2d = Conv2d((1,2))

X = tf.reshape(X, (1, 8, 6, 1))
Y = tf.reshape(Y, (1, 8, 5, 1))
lr = 3e-2

Y_hat = conv2d(X)
for i in range(10):
    with tf.GradientTape(watch_accessed_variables= False) as g:
        g.watch(conv2d.weight)
        Y_hat = conv2d(X)
        l = (abs(Y_hat - Y)) ** 2
        update = tf.multiply(lr, g.gradient(l, conv2d.weight))
        weights = conv2d.get_weights()
        weights = conv2d.weight - update
        conv2d.set_weights(weights)
        if (i + 1) % 2 == 0:
            print(f'epoch {i + 1}, loss {tf.reduce_sum(l):.3f}')